In [2]:
import pandas as pd              # v1.4.3
import numpy as np               # v1.20.1
import matplotlib.pyplot as plt  # v3.5.2
import folium                    # v0.12.1.post1

In [3]:
# Should really use glob to read this all in.
col_types09 = {'DIGEST': 'int16'} # Why did I need this?
Parcels_09 = pd.read_csv(r'F:/GISdata/Atlanta/Tax_Parcels_2009.csv', sep=',',dtype=col_types09 , index_col='OBJECTID', low_memory=False)
Parcels_10 = pd.read_csv(r'F:/GISdata/Atlanta/Tax_Parcels_2010.csv', sep=',',dtype=col_types09 , index_col='OBJECTID', low_memory=False)
col_types = {'TaxYear': 'int16'}
Parcels_11 = pd.read_csv(r'F:/GISdata/Atlanta/Tax_Parcels_2011.csv', sep=',', index_col='OBJECTID', low_memory=False)

In [ ]:
Parcels_12 = pd.read_csv(r'F:/GISdata/Atlanta/Tax_Parcels_2012.csv', sep=',', index_col='OBJECTID', low_memory=False)
Parcels_13 = pd.read_csv(r'F:/GISdata/Atlanta/Tax_Parcels_2013.csv', sep=',', index_col='OBJECTID', low_memory=False)
Parcels_14 = pd.read_csv(r'F:/GISdata/Atlanta/Tax_Parcels_2014.csv', sep=',', index_col='OBJECTID', low_memory=False)
Parcels_15 = pd.read_csv(r'F:/GISdata/Atlanta/Tax_Parcels_2015.csv', sep=',', index_col='OBJECTID', low_memory=False)
Parcels_16 = pd.read_csv(r'F:/GISdata/Atlanta/Tax_Parcels_2016.csv', sep=',', index_col='OBJECTID', dtype=col_types, low_memory=False)
Parcels_17 = pd.read_csv(r'F:/GISdata/Atlanta/Tax_Parcels_2017.csv', sep=',', index_col='OBJECTID', dtype=col_types, low_memory=False)
Parcels_18 = pd.read_csv(r'F:/GISdata/Atlanta/Tax_Parcels_2018.csv', sep=',', index_col='OBJECTID', dtype=col_types, low_memory=False)
Parcels_19 = pd.read_csv(r'F:/GISdata/Atlanta/Tax_Parcels_2019.csv', sep=',', index_col='OBJECTID', dtype=col_types, low_memory=False)
Parcels_20 = pd.read_csv(r'F:/GISdata/Atlanta/Tax_Parcels_2020.csv', sep=',', index_col='OBJECTID', dtype=col_types, low_memory=False)
Parcels_21 = pd.read_csv(r'F:/GISdata/Atlanta/Tax_Parcels_2021.csv', sep=',', index_col='OBJECTID', dtype=col_types, low_memory=False)
Parcels_22 = pd.read_csv(r'F:/GISdata/Atlanta/Tax_Parcels_2022.csv', sep=',', index_col='OBJECTID', dtype=col_types, low_memory=False)

In [4]:
Parcels_09.info(3)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 343897 entries, 1 to 343897
Data columns (total 33 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   DIGEST         343897 non-null  int16  
 1   SITUS          343897 non-null  object 
 2   TAXPIN         343897 non-null  object 
 3   PARID          343897 non-null  object 
 4   ATRPIN         343897 non-null  object 
 5   TAX_DISTR      343897 non-null  object 
 6   OWNER1         343897 non-null  object 
 7   OWNER2         343897 non-null  object 
 8   ADD2           343897 non-null  object 
 9   ADD3           343897 non-null  object 
 10  ADD4           343897 non-null  object 
 11  ADD5           343897 non-null  object 
 12  LUC            343897 non-null  object 
 13  NBHD           343897 non-null  object 
 14  PROP_CLASS     343897 non-null  object 
 15  CLASS          343897 non-null  object 
 16  TOT_APPR       343897 non-null  int64  
 17  TOT_ASSESS     343897 non-nul

In [12]:
# check ['TAXYEAR'] data
Parcels_09.TAXYEAR.head()

OBJECTID
1    2009
2    2009
3    2009
4    2009
5    2009
Name: TAXYEAR, dtype: int64

# The Plan  
- [x] Get property records  
- [ ] Clean data  
    - [x] Fix ['Columns']   
    - [ ] Concat years into single dataframe  
    - [ ] Drop ['Columns'] with lots of NaNs  
    - [ ] Drop rows with lots of NaNs   
    - [ ] Export *.csv
    - [ ] Drop non-ATL properties  
        - [ ] Export ATL-PropertyRecords.csv  

## Parcel data information  
The CAMA information included with the published dataset is selected based on its value to the typical consumer of the data and includes the parcel identification number, the property address, property owner, owner's mailing address, tax district, assessed and appraised value for land and improvements, the number of livable units, acreage, property class and land use class.

Fulton County GIS Collaboration Group [Centerline Address Landmark Data Model](http://share.myfultoncountyga.us/datashare/fultoncounty/Documents/CenterlineAddressLandmarkDataModelDocument.pdf)

In [27]:
# a list of parcel tax IDs in the City of Atlanta.
atlanta_tax_IDs = ['05', '05A', '05B', '05C', '05D', '05E', '05F', '05G', '05H', '05I', '05J',
                    '05K', '05L', '05P', '05Q', '05R', '05S', '05T', '05U', '05V', '05W', '05X',
                  '05Y', '05Z']  

# column names changed in 2011, renaming to make analysis easier
new_cols = {'PARID': 'ParcelID', 'TAX_DISTR': 'TaxDist', 'Digest': 'TaxYear','LIV_UNITS': 'LivUnits', 'OWNER1': 'Owner',
            'TOT_ASSESS': 'TotAssess','TOT_APPR': 'TotAppr', 'TAXYEAR': 'TaxYear'}
Parcels_10.rename(columns=new_cols, inplace=True)
Parcels_09.rename(columns=new_cols, inplace=True)  

# a subset of columns to analyze.
simplified_cols = ['ParcelID', 'TaxDist', 'Owner', 'TotAppr', 'TotAssess', 'LIV_UNITS']

In [19]:
# drop rows with no ['TaxYear'] data
dfs = [Parcels_09, Parcels_10, Parcels_11]#, Parcels_12, Parcels_13, Parcels_14, Parcels_15]
            # , Parcels_16, Parcels_17, Parcels_18, Parcels_19, Parcels_20, Parcels_21, Parcels_22]
for df in dfs:
        df.dropna(subset='TaxYear', inplace=True)
# check that it worked
Parcels_09[pd.isna(Parcels_09['TaxYear'])==True]

,DIGEST,SITUS,TAXPIN,ParcelID,ATRPIN,TaxDist,Owner,OWNER2,ADD2,ADD3,...,STRUCT_YR,TIEBACK,TaxYear,STATUS_COD,LivUnits,PCODE,UNIT_NUM,EXTVER,Shape__Area,Shape__Length
OBJECTID,,,,,,,,,,,,,,,,,,,,,


## [Fulton County Tax Districts](https://fultonassessor.org/wp-content/uploads/sites/16/2018/05/Fulton-County-Tax-Districts-2018.pdf)
05 City of Atlanta  
05A City of Atlanta – Downtown CID  
05B City of Atlanta – Buckhead CID  
05C City of Atlanta – Midtown CID  
05D City of Atlanta – Downtown CID/Midtown CID  
05E City of Atlanta – Downtown CID/Midtown CID/Eastside TAD  
05F City of Atlanta – Downtown CID/Eastside TAD  
05G City of Atlanta – Beltline TAD/Midtown CID  
05H City of Atlanta – Airport West CID  
05I City of Atlanta – Airport West CID/Princeton Lakes TAD  
05J City of Atlanta – Little Five Points CID  
05K City of Atlanta - Westside CID  
05L City of Atlanta – Westside CID/Beltline TAD  
05P City of Atlanta – Stadium TAD  
05Q City of Atlanta – Campbellton TAD  
05R City of Atlanta – Metropolitan TAD  
05S City of Atlanta – Hollowell TAD  
05T City of Atlanta – Beltline TAD  
05U City of Atlanta – Princeton Lakes TAD  
05V City of Atlanta – Perry Bolton TAD  
05W City of Atlanta – Eastside TAD  
05X City of Atlanta – Atlantic Station TAD  
05Y City of Atlanta – Downtown CID/Westside TAD  
05Z City of Atlanta – Westside TAD  
10 City of Alpharetta  
10A City of Alpharetta – North Fulton CID    
15 City of College Park  
15H City of College Park –Airport West CID  
15Z City of College Park – College Park TAD  
20 City of East Point  
20H City of East Point – Airport West CID  
20I City of East Point- East Point TAD #2/Airport West CID  
20J City of East Point – East Point TAD/Airport West CID  
20Y City of East Point – East Point TAD #2  
20Z City of East Point – East Point TAD  
25 City of Fairburn  
25C City of Fairburn – South Fulton CID  
30 City of Hapeville  
30A City of Hapeville – Airport West CID  
30B City of Hapeville – Hapeville TAD/Airport West CID  
30Z City of Hapeville – Hapeville TAD  
35 City of Mountain Park  
40 City of Palmetto  
40C City of Palmetto – South Fulton CID  
45 City of Roswell  
45A City of Roswell – North Fulton CID  
50 City of Union City  
50C City of Union City – South Fulton CID   
50Z City of Union City – Union City TAD  
55 City of South Fulton  
55C City of South Fulton – South Fulton CID  
55D City of South Fulton – Fulton Industrial Boulevard CID  
55H City of South Fulton – Airport West CID  
56 City of Milton  
56A City of Milton – North Fulton CID  
57 City of Johns Creek  
59 City of Sandy Springs  
59B City of Sandy Springs – Fulton Perimeter CID  
65 City of Chattahoochee Hills  
70 Fulton Industrial District  
70D Fulton Industrial District – Fulton Industrial Boulevard CID  
Revised June 9, 2016  

In [20]:
Parcels_11.columns

Index(['ParcelID', 'TaxYear', 'Address', 'AddrNumber', 'AddrPreDir',
       'AddrStreet', 'AddrSuffix', 'AddrPosDir', 'AddrUntTyp', 'AddrUnit',
       'Owner', 'OwnerAddr1', 'OwnerAddr2', 'TaxDist', 'TotAssess',
       'LandAssess', 'ImprAssess', 'TotAppr', 'LandAppr', 'ImprAppr', 'LUCode',
       'ClassCode', 'LivUnits', 'LandAcres', 'NbrHood', 'Subdiv', 'SubdivNum',
       'SubdivLot', 'SubdivBlck', 'Shape__Area', 'Shape__Length'],
      dtype='object')

In [21]:
Parcels_11.count(numeric_only=True)

TaxYear          342983
AddrNumber       342921
TotAssess        342978
LandAssess       342970
ImprAssess       342970
TotAppr          342978
LandAppr         342978
ImprAppr         342970
LivUnits         336732
LandAcres        339442
Shape__Area      342981
Shape__Length    342981
dtype: int64

In [22]:
Parcels_11.shape

(342983, 31)

In [25]:
len(Parcels_11.count(numeric_only=True))

12

## Cleaning Functions

In [ ]:
def check_na(df_list, threshold):
    ''' finds columns with missing data above a threshold value eg; missing more than 100 rows
        and drops them '''
    for df in df_list:
        cols = df.loc[:,df.isna()sum() > threshold].keys()
        df.drop(columns=cols, inplace='True')

### Cleaning the data
Our dats has 31 columns in it  and only 12 have numeric data. Since I plan on concatenating 13 years of data together I'd like to remove as much unnecesary data as possible. I have a list of simplified columns already, which pares it down to just the columns I'm immediately interested in but I'd like to leave my options open in case I uncover an interesting connection that may have links to other columns. I think the right move here is to remove columns that are mostly null values. In the code below we can see that 'AddrPreDir', 'AddrUntTyp', 'AddrUnit', 'Subdiv', 'SubdivNum', 'SubdivLot', and 'SubdivBlck' have drastically less data than the rest of the columns. 

In [35]:
Parcels_11.isna().sum()

ParcelID              0
TaxYear               0
Address               0
AddrNumber           62
AddrPreDir       342775
AddrStreet            3
AddrSuffix        22171
AddrPosDir       207883
AddrUntTyp       317827
AddrUnit         295379
Owner                96
OwnerAddr1          123
OwnerAddr2         1568
TaxDist               0
TotAssess             5
LandAssess           13
ImprAssess           13
TotAppr               5
LandAppr              5
ImprAppr             13
LUCode                5
ClassCode             0
LivUnits           6251
LandAcres          3541
NbrHood               0
Subdiv           341068
SubdivNum        292703
SubdivLot        318682
SubdivBlck       338803
Shape__Area           2
Shape__Length         2
dtype: int64

In [23]:
# find columns with no data or 'na'
parcel_na = Parcels_11.isna().sum()
parcel_na[parcel_na > 50_000]

AddrPreDir    342775
AddrPosDir    207883
AddrUntTyp    317827
AddrUnit      295379
Subdiv        341068
SubdivNum     292703
SubdivLot     318682
SubdivBlck    338803
dtype: int64

In [24]:
parcel_na[parcel_na > 50_000].keys()

Index(['AddrPreDir', 'AddrPosDir', 'AddrUntTyp', 'AddrUnit', 'Subdiv',
       'SubdivNum', 'SubdivLot', 'SubdivBlck'],
      dtype='object')

In [47]:
Parcels_11.loc[:, Parcels_11.isna().sum() > 50_000].columns

Index(['AddrPreDir', 'AddrPosDir', 'AddrUntTyp', 'AddrUnit', 'Subdiv',
       'SubdivNum', 'SubdivLot', 'SubdivBlck'],
      dtype='object')

In [53]:
Parcels_11.loc[:,Parcels_11.isna().sum() > 10_000].keys()

Index(['AddrPreDir', 'AddrSuffix', 'AddrPosDir', 'AddrUntTyp', 'AddrUnit',
       'Subdiv', 'SubdivNum', 'SubdivLot', 'SubdivBlck'],
      dtype='object')

In [26]:
Parcels_09.LivUnits[Parcels_09['LivUnits'] > 100]

OBJECTID
1650      999
2562      242
7471      269
11034     280
11131     101
         ... 
343087    200
343088    200
343089    200
343090    200
343091    200
Name: LivUnits, Length: 603, dtype: int64

In [37]:
Parcels_09.columns

Index(['DIGEST', 'SITUS', 'TAXPIN', 'ParcelID', 'ATRPIN', 'TaxDist', 'Owner',
       'OWNER2', 'ADD2', 'ADD3', 'ADD4', 'ADD5', 'LUC', 'NBHD', 'PROP_CLASS',
       'CLASS', 'TotAppr', 'TotAssess', 'IMPR_APPR', 'LAND_APPR', 'FUL_EX_COD',
       'VAL_ACRES', 'STRUCT_FLR', 'STRUCT_YR', 'TIEBACK', 'TaxYear',
       'STATUS_COD', 'LivUnits', 'PCODE', 'UNIT_NUM', 'EXTVER', 'Shape__Area',
       'Shape__Length'],
      dtype='object')

In [21]:
# Parcels_10.columns

In [22]:
# Parcels_11.columns

In [23]:
# Parcels_12.columns

In [24]:
# Parcels_13.columns

In [25]:
# Parcels_14.columns

In [26]:
# Parcels_15.columns

In [29]:
# Parcels_10.columns

In [53]:
# view of parcels that have an Atlanta Tax ID
''' Still not sure if I want to keep these separate or concat before I analyze. I think I want to concat but then I have to
    do more logic to fine annual data. Here it's already split. I suppose I could just create views onto each year from the 
    main df... hmm.
'''
atlanta_09DF = Parcels_09[Parcels_09['TaxDist'].isin(atlanta_tax_IDs) == True]
atlanta_10DF = Parcels_10[Parcels_10['TaxDist'].isin(atlanta_tax_IDs) == True]
atlanta_11DF = Parcels_11[Parcels_11['TaxDist'].isin(atlanta_tax_IDs) == True]
atlanta_12DF = Parcels_12[Parcels_12['TaxDist'].isin(atlanta_tax_IDs) == True]
atlanta_13DF = Parcels_13[Parcels_13['TaxDist'].isin(atlanta_tax_IDs) == True]
atlanta_14DF = Parcels_14[Parcels_14['TaxDist'].isin(atlanta_tax_IDs) == True]
atlanta_15DF = Parcels_15[Parcels_15['TaxDist'].isin(atlanta_tax_IDs) == True]

## This is markdown ya goof!
year_list = [int(pd.unique(df['TaxYear'])) for df in dfs]  
parcel_data = pd.concat(dfs, keys=[year_list])
parcel_data